# Imports and model functions

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

from scipy.stats import randint, uniform
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score, make_scorer

import time
import psutil
import threading
from memory_profiler import memory_usage

In [3]:
def apply_lgbm(X_train, y_train, best_params=None, n_jobs=15, cv=5):
    """Core LightGBM training with resource monitoring"""
    measurement = {}
    best_params = best_params or {}
    
    lgbm_model = LGBMClassifier(**best_params, verbose=-1, n_jobs=n_jobs)
    cpu_usage = []
    stop_flag = threading.Event()

    def monitor_cpu():
        while not stop_flag.is_set():
            cpu_usage.append(psutil.cpu_percent(interval=0.1))

    try:
        cpu_thread = threading.Thread(target=monitor_cpu)
        cpu_thread.start()
        start_time = time.time()
        
        lgbm_model.fit(X_train, y_train)
        
        training_time = time.time() - start_time
        stop_flag.set()
        cpu_thread.join()

        # Record metrics
        measurement['Training Time (s)'] = training_time
        measurement['Peak CPU (%)'] = max(cpu_usage) if cpu_usage else 0
        measurement['Avg CPU (%)'] = np.mean(cpu_usage) if cpu_usage else 0

        # Cross-validation
        f1_scorer = make_scorer(f1_score, average='weighted')
        cv_scores = cross_val_score(
            lgbm_model, X_train, y_train, cv=cv, scoring=f1_scorer, n_jobs=n_jobs
        )
        
        return cv_scores, measurement, lgbm_model

    except Exception as e:
        print(f"LightGBM training failed: {str(e)}")
        return None, None, None

In [4]:
def eval_dataset_w_LGBM(X_train, X_test, y_train, y_test,
                       params_lgbm={'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': -1}):
    """Evaluation wrapper for LightGBM"""
    cv_scores_lgbm, measurement_lgbm, lgbm_model = apply_lgbm(X_train, y_train, best_params=params_lgbm)
    
    if lgbm_model is None:
        print("⛔ Failed to train LightGBM model")
        return
        
    start_time = time.time()
    y_pred_lgbm = lgbm_model.predict(X_test)
    training_time = time.time() - start_time
    print("Predict Time (s) - ", training_time)

    
    print(f'CV F1: {np.mean(cv_scores_lgbm):.4f} ± {np.std(cv_scores_lgbm):.4f}')
    print(f'Test Accuracy: {accuracy_score(y_test, y_pred_lgbm):.4f}')
    print(classification_report(y_test, y_pred_lgbm, digits=4))
    print("Resource Usage:", measurement_lgbm)

In [5]:
import optuna
from functools import partial

def show_results_LGBM(X_train, X_test, y_train, y_test, n_trials=100):
    def objective(trial, X_train, y_train, cv=5):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 15),
            'num_leaves': trial.suggest_int('num_leaves', 10, 100),
            'subsample': trial.suggest_float('subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True)
        }
        
        cv_scores, _, model = apply_lgbm(X_train, y_train, best_params=params, cv=cv)
        return np.mean(cv_scores) if cv_scores is not None else 0

    study = optuna.create_study(direction='maximize')
    study.optimize(partial(objective, X_train=X_train, y_train=y_train), n_trials=n_trials)
    best_params = study.best_params

    cv_scores_lgbm, measurement_lgbm, lgbm_model = apply_lgbm(X_train, y_train, best_params=best_params)
    
    if cv_scores_lgbm is None:
        print("LightGBM training failed")
        return

    y_pred_lgbm = lgbm_model.predict(X_test)

    try:
        f1 = f1_score(y_test, y_pred_lgbm, average='weighted')
        accuracy = accuracy_score(y_test, y_pred_lgbm)
        
        print("\nLightGBM Evaluation Results:")
        print("-" * 50)
        print(f'CV F1: {np.mean(cv_scores_lgbm):.4f} ± {np.std(cv_scores_lgbm):.4f}')
        print(f'Test F1: {f1:.4f}')
        print(f'Test Accuracy: {accuracy:.4f}')
        print("\nResource Usage:", measurement_lgbm)
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred_lgbm))
    
    except Exception as e:
        print(f"Error in LightGBM evaluation: {str(e)}")
        raise
    
    return lgbm_model, best_params

# Prep for model training cicids2017

In [5]:
# Reading data
df = pd.read_csv("..\..\data prep\cicids2017_prep\cicids2017_42feat_97percent.csv")

In [6]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, BorderlineSMOTE

In [7]:
# Preparing training and test splits
X = df.drop('Attack Type', axis=1)
y = df['Attack Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [8]:
# Initialize scaling algos
MMS = MinMaxScaler()
X_train_MMS_scaled = MMS.fit_transform(X_train)
X_test_MMS_scaled = MMS.transform(X_test)

In [9]:
X_train_scaled_rus_MMS, y_train_scaled_rus_MMS = RandomUnderSampler(sampling_strategy={'Normal Traffic': 500000}, random_state=42).fit_resample(X_train_MMS_scaled, y_train)

In [10]:
X_train_resampled_scaled_MMS_SMOTE, y_train_resampled_scaled_MMS_SMOTE = SMOTE(sampling_strategy={'Bots': 7500, 'Web Attacks': 7500, 'Brute Force': 7000, 'Port Scanning': 70000, 'DDoS':90000, 'DoS': 200000}, random_state=42).fit_resample(X_train_scaled_rus_MMS, y_train_scaled_rus_MMS)

# Sync classes

In [11]:
# Function to combine classes
def combine_classes(y, class_mapping):
    return y.map(class_mapping)
# Define the mapping
class_mapping = {
    'Web Attacks': 'Other',
    'Port Scanning': 'Other',
    'Normal Traffic': 'Normal Traffic',
    'Bots': 'Bots',
    'Brute Force': 'Brute Force',
    'DDoS': 'DDoS',
    'DoS': 'DoS'
}

In [12]:
df["Attack Type"].unique()

array(['Normal Traffic', 'DDoS', 'Port Scanning', 'Bots', 'Web Attacks',
       'Brute Force', 'DoS'], dtype=object)

In [13]:
# Apply to all your sets
y_train = combine_classes(y_train, class_mapping)
y_test = combine_classes(y_test, class_mapping)

y_train_scaled_rus_MMS = combine_classes(y_train_scaled_rus_MMS, class_mapping)
y_train_resampled_scaled_MMS_SMOTE = combine_classes(y_train_resampled_scaled_MMS_SMOTE, class_mapping)

# Search best params for MMS SMOTE

In [15]:
lbgm_model, best_params = show_results_LGBM(X_train_resampled_scaled_MMS_SMOTE, 
                                    X_test_MMS_scaled,
                                    y_train_resampled_scaled_MMS_SMOTE, 
                                    y_test, 
                                    n_trials=60)

[I 2025-05-03 22:36:28,992] A new study created in memory with name: no-name-080196f8-38b6-46ab-879c-c139e2291ccb
[I 2025-05-03 22:39:13,512] Trial 0 finished with value: 0.9933464593808505 and parameters: {'n_estimators': 326, 'learning_rate': 0.1659552439902487, 'max_depth': 6, 'num_leaves': 92, 'subsample': 0.6743862447023563, 'colsample_bytree': 0.794395512937653, 'reg_alpha': 3.4258989444440786, 'reg_lambda': 1.3482092208482432}. Best is trial 0 with value: 0.9933464593808505.
[I 2025-05-03 22:44:41,718] Trial 1 finished with value: 0.9883197926246972 and parameters: {'n_estimators': 511, 'learning_rate': 0.0018982673862374551, 'max_depth': 9, 'num_leaves': 95, 'subsample': 0.7278099903937385, 'colsample_bytree': 0.9314858293645412, 'reg_alpha': 0.0062344139435361965, 'reg_lambda': 1.2684686992373289}. Best is trial 0 with value: 0.9933464593808505.
[I 2025-05-03 22:46:25,355] Trial 2 finished with value: 0.9889686355369914 and parameters: {'n_estimators': 153, 'learning_rate': 0.


LightGBM Evaluation Results:
--------------------------------------------------
CV F1: 0.9939 ± 0.0106
Test F1: 0.9990
Test Accuracy: 0.9990

Resource Usage: {'Training Time (s)': 32.61628818511963, 'Peak CPU (%)': 75.2, 'Avg CPU (%)': 53.185139318885454}

Classification Report:
                precision    recall  f1-score   support

          Bots       0.69      0.95      0.80       584
   Brute Force       1.00      1.00      1.00      2745
          DDoS       1.00      1.00      1.00     38404
           DoS       1.00      1.00      1.00     58124
Normal Traffic       1.00      1.00      1.00    628518
         Other       0.99      1.00      0.99     27851

      accuracy                           1.00    756226
     macro avg       0.95      0.99      0.97    756226
  weighted avg       1.00      1.00      1.00    756226



In [22]:
eval_dataset_w_LGBM(X_train_resampled_scaled_MMS_SMOTE, X_test_MMS_scaled, y_train_resampled_scaled_MMS_SMOTE, y_test, params_lgbm={'n_estimators': 228, 'learning_rate': 0.07241523535942174, 'max_depth': 14, 'num_leaves': 79, 'subsample': 0.5650088660864082, 'colsample_bytree': 0.8850730957587873, 'reg_alpha': 0.31650105405212536, 'reg_lambda': 3.8724602641849213})

Predict Time (s) -  9.027400255203247
CV F1: 0.9939 ± 0.0106
Test Accuracy: 0.9990
                precision    recall  f1-score   support

          Bots     0.6898    0.9521    0.8000       584
   Brute Force     0.9993    0.9989    0.9991      2745
          DDoS     0.9998    0.9998    0.9998     38404
           DoS     0.9977    0.9998    0.9987     58124
Normal Traffic     0.9999    0.9989    0.9994    628518
         Other     0.9889    0.9994    0.9941     27851

      accuracy                         0.9990    756226
     macro avg     0.9459    0.9915    0.9652    756226
  weighted avg     0.9991    0.9990    0.9990    756226

Resource Usage: {'Training Time (s)': 38.28263568878174, 'Peak CPU (%)': 99.1, 'Avg CPU (%)': 74.17698412698412}


# Binary with cross-val between datasets

## Prepare data

In [16]:
# Reading data
df = pd.read_csv("..\..\CrossVal between datasets\cicids2017_training.csv")

X_train = df.drop('Attack Type', axis=1)
y_train = df['Attack Type']

del df

In [17]:
# Reading data
df = pd.read_csv("..\..\CrossVal between datasets\cicids2018_test_of_2017.csv")

X_test = df.drop('Attack Type', axis=1)
y_test = df['Attack Type']

del df

In [18]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, BorderlineSMOTE

In [19]:
# Initialize scaling algos
MMS = MinMaxScaler()
X_train_MMS_scaled = MMS.fit_transform(X_train)
X_test_MMS_scaled = MMS.transform(X_test)

In [20]:
X_train_scaled_rus_MMS, y_train_scaled_rus_MMS = RandomUnderSampler(sampling_strategy={'Normal Traffic': 650000}, random_state=42).fit_resample(X_train_MMS_scaled, y_train)

In [21]:
X_train_resampled_scaled_MMS_SMOTE, y_train_resampled_scaled_MMS_SMOTE = SMOTE(sampling_strategy={'Bots': 10000, 'Web Attacks': 10000, 'Brute Force': 10000, 'Port Scanning': 91000, 'DDoS': 130000, 'DoS': 200000}, random_state=42).fit_resample(X_train_scaled_rus_MMS, y_train_scaled_rus_MMS)

In [22]:
group_mapping_2017 = {
    'Normal Traffic': 'BENIGN',
    'DoS': 'Attack',
    'DDoS': 'Attack',
    'Brute Force': 'Attack',
    'Bots': 'Attack',
    'Web Attacks': 'Attack',
    'Port Scanning': 'Attack'}

In [23]:
group_mapping_2018 = {
    'Normal Traffic': 'BENIGN',
    'DoS': 'Attack',
    'DDoS': 'Attack',
    'Brute Force': 'Attack',
    'Bots': 'Attack',
    'Other': 'Attack'}

In [24]:
y_train_resampled_scaled_MMS_SMOTE = y_train_resampled_scaled_MMS_SMOTE.map(group_mapping_2017)
y_test = y_test.map(group_mapping_2018)

## Search Parameters

In [76]:
lbgm_model, best_params = show_results_LGBM(X_train_resampled_scaled_MMS_SMOTE, 
                                    X_test_MMS_scaled,
                                    y_train_resampled_scaled_MMS_SMOTE, 
                                    y_test, 
                                    n_trials=60)

[I 2025-05-08 15:52:45,745] A new study created in memory with name: no-name-f6c15b4c-ba73-4338-94d7-8d68efd3f5c4
[I 2025-05-08 15:53:20,941] Trial 0 finished with value: 0.9463691980440976 and parameters: {'n_estimators': 236, 'learning_rate': 0.0031743583016229497, 'max_depth': 5, 'num_leaves': 10, 'subsample': 0.6599208139910135, 'colsample_bytree': 0.9866917940844662, 'reg_alpha': 0.008200707779018225, 'reg_lambda': 3.924635944296941}. Best is trial 0 with value: 0.9463691980440976.
[I 2025-05-08 15:54:15,987] Trial 1 finished with value: 0.9947781486089827 and parameters: {'n_estimators': 251, 'learning_rate': 0.05653593901676786, 'max_depth': 7, 'num_leaves': 61, 'subsample': 0.8930974500057347, 'colsample_bytree': 0.9318301386899517, 'reg_alpha': 0.0018429557707955522, 'reg_lambda': 0.13314839100032558}. Best is trial 1 with value: 0.9947781486089827.
[I 2025-05-08 15:56:22,370] Trial 2 finished with value: 0.9962936879063552 and parameters: {'n_estimators': 717, 'learning_rate'


LightGBM Evaluation Results:
--------------------------------------------------
CV F1: 0.9975 ± 0.0030
Test F1: 0.8548
Test Accuracy: 0.8818

Resource Usage: {'Training Time (s)': 17.43651819229126, 'Peak CPU (%)': 100.0, 'Avg CPU (%)': 94.97647058823527}

Classification Report:
              precision    recall  f1-score   support

      Attack       0.99      0.31      0.47   2746847
      BENIGN       0.88      1.00      0.93  13390249

    accuracy                           0.88  16137096
   macro avg       0.93      0.65      0.70  16137096
weighted avg       0.90      0.88      0.85  16137096



In [ ]:
eval_dataset_w_LGBM(X_train_resampled_scaled_MMS_SMOTE, X_test_MMS_scaled, y_train_resampled_scaled_MMS_SMOTE, y_test, params_lgbm={'n_estimators': 285, 'learning_rate': 0.15274828247019778, 'max_depth': 5, 'num_leaves': 44, 'subsample': 0.6210331060028171, 'colsample_bytree': 0.9909317475119969, 'reg_alpha': 0.4761330684134722, 'reg_lambda': 1.8813380938652553})

Predict Time (s) -  18.106565952301025
CV F1: 0.9974 ± 0.0031
Test Accuracy: 0.8844
              precision    recall  f1-score   support

      Attack     0.9908    0.3237    0.4880   2746847
      BENIGN     0.8781    0.9994    0.9348  13390249

    accuracy                         0.8844  16137096
   macro avg     0.9345    0.6616    0.7114  16137096
weighted avg     0.8973    0.8844    0.8588  16137096

Resource Usage: {'Training Time (s)': 7.788529872894287, 'Peak CPU (%)': 100.0, 'Avg CPU (%)': 96.71780821917808}


# Binary with cross-val on single dataset

## Prepare data

In [77]:
# Reading data
df = pd.read_csv("..\..\CrossVal between datasets\cicids2017_training.csv")

# Preparing training and test splits
X = df.drop('Attack Type', axis=1)
y = df['Attack Type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [78]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, BorderlineSMOTE

In [79]:
# Initialize scaling algos
MMS = MinMaxScaler()
X_train_MMS_scaled = MMS.fit_transform(X_train)
X_test_MMS_scaled = MMS.transform(X_test)

In [80]:
X_train_scaled_rus_MMS, y_train_scaled_rus_MMS = RandomUnderSampler(sampling_strategy={'Normal Traffic': 650000}, random_state=42).fit_resample(X_train_MMS_scaled, y_train)

In [81]:
X_train_resampled_scaled_MMS_SMOTE, y_train_resampled_scaled_MMS_SMOTE = SMOTE(sampling_strategy={'Bots': 10000, 'Web Attacks': 10000, 'Brute Force': 10000, 'Port Scanning': 91000, 'DDoS': 130000, 'DoS': 200000}, random_state=42).fit_resample(X_train_scaled_rus_MMS, y_train_scaled_rus_MMS)

In [82]:
group_mapping_2017 = {
    'Normal Traffic': 'BENIGN',
    'DoS': 'Attack',
    'DDoS': 'Attack',
    'Brute Force': 'Attack',
    'Bots': 'Attack',
    'Web Attacks': 'Attack',
    'Port Scanning': 'Attack'}

In [83]:
y_train_resampled_scaled_MMS_SMOTE = y_train_resampled_scaled_MMS_SMOTE.map(group_mapping_2017)
y_test = y_test.map(group_mapping_2017)

## Search Parameters

In [84]:
lbgm_model, best_params = show_results_LGBM(X_train_resampled_scaled_MMS_SMOTE, 
                                    X_test_MMS_scaled,
                                    y_train_resampled_scaled_MMS_SMOTE, 
                                    y_test, 
                                    n_trials=60)

[I 2025-05-08 17:19:27,084] A new study created in memory with name: no-name-ca30a774-9e69-4742-8e88-1da0b89a4147
[I 2025-05-08 17:20:06,623] Trial 0 finished with value: 0.9984349205058625 and parameters: {'n_estimators': 383, 'learning_rate': 0.12111930457526512, 'max_depth': 3, 'num_leaves': 91, 'subsample': 0.5835162686194524, 'colsample_bytree': 0.6888231079926798, 'reg_alpha': 0.05721460438284763, 'reg_lambda': 0.001587734342698104}. Best is trial 0 with value: 0.9984349205058625.
[I 2025-05-08 17:21:30,157] Trial 1 finished with value: 0.9980886755221992 and parameters: {'n_estimators': 332, 'learning_rate': 0.009589780908912389, 'max_depth': 15, 'num_leaves': 96, 'subsample': 0.5405799783730312, 'colsample_bytree': 0.7626151265013338, 'reg_alpha': 1.9556808899672304, 'reg_lambda': 1.403980100280639}. Best is trial 0 with value: 0.9984349205058625.
[I 2025-05-08 17:22:24,576] Trial 2 finished with value: 0.998981839357992 and parameters: {'n_estimators': 437, 'learning_rate': 0.


LightGBM Evaluation Results:
--------------------------------------------------
CV F1: 0.9991 ± 0.0003
Test F1: 0.9990
Test Accuracy: 0.9990

Resource Usage: {'Training Time (s)': 11.896596431732178, 'Peak CPU (%)': 98.3, 'Avg CPU (%)': 92.89913793103446}

Classification Report:
              precision    recall  f1-score   support

      Attack       0.99      1.00      1.00    127708
      BENIGN       1.00      1.00      1.00    628518

    accuracy                           1.00    756226
   macro avg       1.00      1.00      1.00    756226
weighted avg       1.00      1.00      1.00    756226



In [85]:
eval_dataset_w_LGBM(X_train_resampled_scaled_MMS_SMOTE, X_test_MMS_scaled, y_train_resampled_scaled_MMS_SMOTE, y_test, params_lgbm={'n_estimators': 285, 'learning_rate': 0.15274828247019778, 'max_depth': 5, 'num_leaves': 44, 'subsample': 0.6210331060028171, 'colsample_bytree': 0.9909317475119969, 'reg_alpha': 0.4761330684134722, 'reg_lambda': 1.8813380938652553})

Predict Time (s) -  0.9719040393829346
CV F1: 0.9990 ± 0.0004
Test Accuracy: 0.9990
              precision    recall  f1-score   support

      Attack     0.9944    0.9998    0.9971    127708
      BENIGN     1.0000    0.9989    0.9994    628518

    accuracy                         0.9990    756226
   macro avg     0.9972    0.9993    0.9982    756226
weighted avg     0.9990    0.9990    0.9990    756226

Resource Usage: {'Training Time (s)': 6.1358442306518555, 'Peak CPU (%)': 100.0, 'Avg CPU (%)': 91.40338983050847}


# MultiClass with cross-val between datasets

## Prepare data

In [6]:
# Reading data
df = pd.read_csv("..\..\CrossVal between datasets\cicids2017_training.csv")

X_train = df.drop('Attack Type', axis=1)
y_train = df['Attack Type']

del df

In [7]:
# Reading data
df = pd.read_csv("..\..\CrossVal between datasets\cicids2018_test_of_2017.csv")

X_test = df.drop('Attack Type', axis=1)
y_test = df['Attack Type']

del df

In [8]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, BorderlineSMOTE

In [9]:
# Initialize scaling algos
MMS = MinMaxScaler()
X_train_MMS_scaled = MMS.fit_transform(X_train)
X_test_MMS_scaled = MMS.transform(X_test)

In [10]:
X_train_scaled_rus_MMS, y_train_scaled_rus_MMS = RandomUnderSampler(sampling_strategy={'Normal Traffic': 650000}, random_state=42).fit_resample(X_train_MMS_scaled, y_train)

In [11]:
X_train_resampled_scaled_MMS_SMOTE, y_train_resampled_scaled_MMS_SMOTE = SMOTE(sampling_strategy={'Bots': 10000, 'Web Attacks': 10000, 'Brute Force': 10000, 'Port Scanning': 91000, 'DDoS': 130000, 'DoS': 200000}, random_state=42).fit_resample(X_train_scaled_rus_MMS, y_train_scaled_rus_MMS)

In [12]:
group_mapping_2017 = {
    'Normal Traffic': 'Normal Traffic',
    'DoS': 'DoS',
    'DDoS': 'DDoS',
    'Brute Force': 'Brute Force',
    'Bots': 'Bots',
    'Web Attacks': 'Other',
    'Port Scanning': 'Other'}

In [13]:
y_train_resampled_scaled_MMS_SMOTE = y_train_resampled_scaled_MMS_SMOTE.map(group_mapping_2017)

## Search Parameters

In [14]:
lbgm_model, best_params = show_results_LGBM(X_train_resampled_scaled_MMS_SMOTE, 
                                    X_test_MMS_scaled,
                                    y_train_resampled_scaled_MMS_SMOTE, 
                                    y_test, 
                                    n_trials=60)

[I 2025-05-08 23:53:20,967] A new study created in memory with name: no-name-6afee2c1-b922-4599-a198-d2e28b069daa
[I 2025-05-08 23:57:02,242] Trial 0 finished with value: 0.9840412331819719 and parameters: {'n_estimators': 512, 'learning_rate': 0.2092436753918431, 'max_depth': 9, 'num_leaves': 16, 'subsample': 0.9561399934438196, 'colsample_bytree': 0.7416762250700377, 'reg_alpha': 4.2621481829676755, 'reg_lambda': 5.597727225506113}. Best is trial 0 with value: 0.9840412331819719.
[I 2025-05-08 23:59:24,288] Trial 1 finished with value: 0.9795937113057958 and parameters: {'n_estimators': 176, 'learning_rate': 0.13043822726196597, 'max_depth': 13, 'num_leaves': 90, 'subsample': 0.9487989713954219, 'colsample_bytree': 0.625323698865064, 'reg_alpha': 0.3137953126939265, 'reg_lambda': 0.07441513111476677}. Best is trial 0 with value: 0.9840412331819719.
[I 2025-05-09 00:07:57,741] Trial 2 finished with value: 0.9713474562224054 and parameters: {'n_estimators': 805, 'learning_rate': 0.0013

KeyboardInterrupt: 

In [15]:
eval_dataset_w_LGBM(X_train_resampled_scaled_MMS_SMOTE, X_test_MMS_scaled, y_train_resampled_scaled_MMS_SMOTE, y_test, params_lgbm={'n_estimators': 475, 'learning_rate': 0.058716115604362774, 'max_depth': 12, 'num_leaves': 11, 'subsample': 0.5057428968514667, 'colsample_bytree': 0.8101229173073474, 'reg_alpha': 0.006541575882940611, 'reg_lambda': 0.02825941211887404})

Predict Time (s) -  189.2470042705536
CV F1: 0.9874 ± 0.0234
Test Accuracy: 0.8614


f:\!STUDY\1TMO\4 Сем\ДИПЛОМ\CODE\NN-env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
f:\!STUDY\1TMO\4 Сем\ДИПЛОМ\CODE\NN-env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

          Bots     0.1875    0.0000    0.0000    286191
   Brute Force     0.9568    0.2455    0.3907    381784
          DDoS     0.0000    0.0000    0.0000   1263933
           DoS     0.9962    0.7114    0.8301    654300
Normal Traffic     0.8793    0.9944    0.9333  13390249
         Other     0.0580    0.1545    0.0843    160639

      accuracy                         0.8614  16137096
     macro avg     0.5130    0.3510    0.3731  16137096
  weighted avg     0.7965    0.8614    0.8182  16137096

Resource Usage: {'Training Time (s)': 1089.6048467159271, 'Peak CPU (%)': 100.0, 'Avg CPU (%)': 99.99891181988744}


f:\!STUDY\1TMO\4 Сем\ДИПЛОМ\CODE\NN-env\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
